In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [2]:
os.makedirs("snapshots", exist_ok=True)

In [3]:
#!wget --header 'Host: doc-14-58-docs.googleusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://drive.google.com/uc?id=1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R' --header 'DNT: 1' --header 'Cookie: AUTH_du2s671ksr3d23d9f5l3dgdt8geo7lbv_nonce=diq6am7vgd4oc; _ga=GA1.2.1544767178.1559556380' --header 'Upgrade-Insecure-Requests: 1' 'https://doc-14-58-docs.googleusercontent.com/docs/securesc/tjmqopet735sj103qll72jkoabtv3hkc/6fn10j96e3urpc7nkc5hbss73ougqi0f/1587469725000/01328360181817234523/06581179416702578279/1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R?e=download&authuser=0&nonce=diq6am7vgd4oc&user=06581179416702578279&hash=bc51kf9i7kh6r6em4839g163n1lsjf43' --output-document 'resnet50_csv_10.h5'

In [4]:
# path/to/image.jpg,x1,y1,x2,y2,class_name
#required for keras retina net
df =pd.read_csv('traindata.csv')

In [5]:
df.head()

,imagenames,topx,topy,bottomx,bottomy
0,Images/car1.jpeg,582,274,700,321
1,Images/car2.jpeg,80,382,288,449
2,Images/car3.jpeg,41,219,115,260
3,Images/car4.jpeg,217,313,359,359
4,Images/car5.jpeg,209,220,866,554


In [6]:
df['class_name'] = 'license_plate'

In [7]:
df.head()

,imagenames,topx,topy,bottomx,bottomy,class_name
0,Images/car1.jpeg,582,274,700,321,license_plate
1,Images/car2.jpeg,80,382,288,449,license_plate
2,Images/car3.jpeg,41,219,115,260,license_plate
3,Images/car4.jpeg,217,313,359,359,license_plate
4,Images/car5.jpeg,209,220,866,554,license_plate


In [8]:
train_df, test_df = train_test_split(
  df,
  test_size=0.2,
  random_state=RANDOM_SEED
)

In [16]:
annotations_file = 'annotations.csv'
classes_file = 'classes.csv'

In [18]:
train_df.to_csv(annotations_file, index=False, header=None)


In [19]:
#writing a classes file

classes = set(['license_plate'])

with open(classes_file, 'w') as f:
    
    for i, line in enumerate(sorted(classes)):
        f.write('{},{}\n'.format(line,i))

In [20]:
os.makedirs("snapshots", exist_ok=True)

In [21]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [27]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
##Run the below command from terminal

In [28]:
!retinanet-train --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 32 --steps 500 --epochs 20 csv annotations.csv classes.csv

Using TensorFlow backend.
2020-04-21 12:39:26.517154: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-04-21 12:39:26.518850: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Creating model, this may take a second...
2020-04-21 12:39:27.348154: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-21 12:39:27.359311: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 4f5b:00:00.0 name: Tesla M60 computeCapability: 5.2
coreClock: 1.1775GHz coreCount: 16 deviceMemorySize: 7.94GiB deviceMemoryBandwidth: 149.31GiB/s
2020-04-21 12:39:27.360168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 1 with properties: 
pciBusID: 7f4d:00:00.0 name: Tesla M60 computeCapability: 5.2
coreClock: 1.1775GHz coreCount: 16 devic

2020-04-21 12:39:28.670360: F ./tensorflow/core/kernels/random_op_gpu.h:232] Non-OK-status: GpuLaunchKernel(FillPhiloxRandomKernelLaunch<Distribution>, num_blocks, block_size, 0, d.stream(), gen, data, size, dist) status: Internal: out of memory
Aborted (core dumped)
